# Fabric Scanner API to capture all Fabric artifcats' grants and access information

The steps below provide details to have a Service principal read the Fabric metadata. Currently, there is no way in Fabric to show what artifacts are shared with whom. We will capture this metadata via the Scanner APIs

https://learn.microsoft.com/en-us/fabric/governance/metadata-scanning-overview

Prerequisites:
- Follow this documentation to enable SP authentication for read-only Admin APIs
    https://learn.microsoft.com/en-us/fabric/admin/metadata-scanning-enable-read-only-apis
- 	The APIs we are going to use are GetModifiedWorkspaces, WorkspacegetInfo, WorkspacescanStatus and WorkspacescanResult
- See this documentation for further guidance -

    https://learn.microsoft.com/en-us/fabric/admin/metadata-scanning-setup
    
    https://learn.microsoft.com/en-us/fabric/governance/metadata-scanning-run


# Notebook overview

This notebook captures Fabric artifact user grants and accesses via Scanner APIs, transforms them into a  standard format (as each artifact's metadata output is not necessarily consistent). We are storing the final output in a table in the lakehouse and create Power BI reports to provide the relevant information

Future Enhancements planned:
- Divide list of workspace ids in chunks of 100 workspaces and execute each chunk
- Implement incremental scan

<mark>**Please ensure notebook setup is done in the next 2 cells before running the notebook**</mark>

In [38]:
# Enter details of workspace, lakehouse where the final table called 'WorkspaceAccessMetadata' should reside in
myLakehouse = 'lakehouse01'
myWorkspace = 'WS_SagarFabric01'
myTablePath = "abfss://"+myWorkspace+"@onelake.dfs.fabric.microsoft.com/"+myLakehouse+".Lakehouse/Tables/WorkspaceAccessMetadata"
print(myTablePath)

StatementMeta(, 6e945aa3-363c-4cdf-9289-2cbdc3e9199b, 40, Finished, Available)

abfss://WS_SagarFabric01@onelake.dfs.fabric.microsoft.com/lakehouse01.Lakehouse/Tables/WorkspaceAccessMetadata


In [39]:
# Read secrets from Azure Key Vault
# Ensure the following items are defined in the keyvault before running the notebook
key_vault = "https://pvlab-9a758f-keyvault.vault.azure.net/"
client_secret = mssparkutils.credentials.getSecret(key_vault , "secret-app-Fabric-Power-API")
tenant = mssparkutils.credentials.getSecret(key_vault , "FabricTenantId")
client = mssparkutils.credentials.getSecret(key_vault , "app-Fabric-Power-API-ClientID")
#print(tenant)
#print(client)

StatementMeta(, 6e945aa3-363c-4cdf-9289-2cbdc3e9199b, 41, Finished, Available)

In [40]:
# Currently this notebook does not support incremental metadata updates. Drop the final metadata table each time this notebook is executed
if spark.catalog.tableExists("WorkspaceAccessMetadata"):
    drop_stmt = 'DROP TABLE '+myLakehouse+'.WorkspaceAccessMetadata'
    result = spark.sql(drop_stmt)
    

StatementMeta(, 6e945aa3-363c-4cdf-9289-2cbdc3e9199b, 42, Finished, Available)

In [41]:
# Authentication - Replace string variables with your relevant values      
import json, requests, pandas as pd
import datetime
 
try:
    from azure.identity import ClientSecretCredential
except Exception:
     !pip install azure.identity
     from azure.identity import ClientSecretCredential

authority_url= f'https://login.microsoftonline.com/'
#print(authority_url)

# Generates the access token for the Service Principal
api = 'https://analysis.windows.net/powerbi/api/.default'
auth = ClientSecretCredential(authority = authority_url,
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
access_token = auth.get_token(api)
access_token = access_token.token

#print(access_token) 
print('\nSuccessfully authenticated.')   

StatementMeta(, 6e945aa3-363c-4cdf-9289-2cbdc3e9199b, 43, Finished, Available)


Successfully authenticated.


In [42]:
# Get workspace ids
base_url = 'https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified?excludePersonalWorkspaces=True&excludeInActiveWorkspaces=True'
# base_url = 'https://api.powerbi.com/v1.0/myorg/admin/workspaces/modified'
# base_url = 'https://api.powerbi.com/v1.0/myorg/admin/workspaces/getInfo'
header = {'Authorization': f'Bearer {access_token}'}

response = requests.get(base_url, headers=header)
#print(response.content)
#print(response['id'][0])
data=response.json()
df_workpaceids = pd.DataFrame.from_dict(data) 
display('Workspaceid: '+ df_workpaceids['id'])

StatementMeta(, 6e945aa3-363c-4cdf-9289-2cbdc3e9199b, 44, Finished, Available)

0    Workspaceid: 4b63d48a-82f6-4f27-bb75-7a1b8b22df7e
1    Workspaceid: edf1fd48-b561-4540-82fc-37aed98c4963
2    Workspaceid: e356b66e-9a99-49a9-886f-4a6055a26644
3    Workspaceid: 29e068aa-3094-4347-9afd-30c8167ba35e
4    Workspaceid: bb6157b3-92fb-4995-a2ae-c511bd00b75b
5    Workspaceid: 6da79f32-7624-4815-a64c-325ea59b39d4
6    Workspaceid: 988d5ee5-51ff-4c8f-867a-7841d6b96b1e
7    Workspaceid: 7e8019c5-9879-4ee7-adc6-99ce5081feee
8    Workspaceid: e26a57cd-fa71-4b01-8ea3-6190d4be4369
Name: id, dtype: object

In [43]:
import time

# Loop through all workspace ids
for index, row in df_workpaceids.iterrows():
    workspaceid = row['id']
    print('Workspaceid: ' + workspaceid)

    # Trigger workspace scan call for each workspace id
    base_url = 'https://api.powerbi.com/v1.0/myorg/admin/workspaces/getInfo'
    header = {"Content-Type": "application/json","Authorization": f'Bearer {access_token}'}
    payload = json.dumps({"workspaces":[workspaceid]})
    parametersurl="?getArtifactUsers=true"
    response = requests.request("POST", base_url+parametersurl, headers=header, data=payload)
    #print(response.text)

    # Get the scan id
    data=response.json()
    df_getInfo = pd.DataFrame.from_dict([data]) 
    #display(df_getInfo)
    #display(df_getInfo['data']['id'])
    scanid = df_getInfo['id'][0]
    display('Scanid for Workspaceid ' + workspaceid + ' is ' +scanid) 

    
    # Check if the scan is complete
    base_url = 'https://api.powerbi.com/v1.0/myorg/admin/workspaces/scanStatus/'
    header = {"Authorization": f'Bearer {access_token}'}
    # scanid=df_getInfo['id'][0]
    response = requests.get(base_url+scanid, headers=header)
    data=response.json()
    df_scanStatus = pd.DataFrame.from_dict([data]) 
    scanStatus = df_scanStatus['status'][0]
    display('Status for Scanid ' + scanid + ' for Workspaceid ' + workspaceid + ' is ' +scanStatus) 

    # keep checking for scan status=succeeded every 30 seconds
    while (scanStatus != 'Succeeded'):
        time.sleep(30)
        response = requests.get(base_url+scanid, headers=header)
        data=response.json()
        df_scanStatus = pd.DataFrame.from_dict([data]) 
        scanStatus = df_scanStatus['status'][0]
        display('Status for Scanid ' + scanid + ' for Workspaceid ' + workspaceid + ' is ' +scanStatus) 

    display('Scan for Scanid: '+ scanid + ' is complete')

    # Get the metadata for each scan
    base_url = 'https://api.powerbi.com/v1.0/myorg/admin/workspaces/scanResult/'
    header = {"Authorization": f'Bearer {access_token}'}
    #scanid=df_getInfo['id'][0]

    response = requests.get(base_url+scanid, headers=header)

    #Convert the JSON data into a dataframe
    jsondata = json.loads(json.dumps(response.text))
    df_scanResults = spark.read.json(sc.parallelize([jsondata]))
    #display(df_scanResults)

    # load scan results into multiple views, perform transformations and load final output to a table
    df_scanResults.createOrReplaceTempView("ScannerAPIoutput_view_0")
    #df_scanResults.write.format("delta").mode("append").save("abfss://WS_SagarFabric01@onelake.dfs.fabric.microsoft.com/users01.Lakehouse/Tables/FabricScannerAPI_raw")

    # Explode the content 
    result = spark.sql("CREATE OR REPLACE TEMP VIEW ScannerAPIoutput_view AS select explode(workspaces) AS workspaces from ScannerAPIoutput_view_0")

    result = spark.sql("Create or replace TEMP VIEW WorkspaceMetadata_view AS select workspaces.* from ScannerAPIoutput_view")

    df_workspaceName = spark.sql("select name from WorkspaceMetadata_view")
    #display(df_workspaceName)
    workspaceName = df_workspaceName.first()[0]
    #display(workspaceName)

    # Include only user defined workspaces
    if ('Fabric Capacity Metrics' in workspaceName) or ('Premium Capacity Utilization' in workspaceName) or ('Admin monitoring' in workspaceName):
        pass
    else:
        print ('Valid workspace - do the processing')
        #result = spark.sql("select * from WorkspaceMetadata_view")
        #display(result)

        # check if Notebook metadata is present in the workspace
        df_viewcolumns = spark.sql("select * from WorkspaceMetadata_view LIMIT 1")
        #display(df_viewcolumns)

        # The code below is a series of if statements to look for presence of metadata for all Fabric artifacts such as Notebooks, reports, Lakehouses etc.
        # It checks is metadata is present for these artifacts and it is, transforms them and finally loads to a table
        if ("Notebook" not in df_viewcolumns.columns):
            pass
        else:
            print('notebook column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["Notebook"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('notebook column has value')    

                    # explode Notebook metadata
        	
                    result = spark.sql("""
                                create or replace temp view NotebookMetadata_view AS
                                SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                                isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                                explode(Notebook) AS NoteBookMetadata FROM WorkspaceMetadata_view
                              """)

                    # Create uniform format
                    result = spark.sql("""
                                create or replace temp view NotebookUserAccess_view AS 
                                Select Notebookmetadata_view.*, NotebookMetadata.name AS ArtifactName, explode(NotebookMetadata.users) as NotebookUserAccess from Notebookmetadata_view
                              """)

                    result = spark.sql("""
                                SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                                WorkspaceState,'Notebook' AS ArtifactType, ArtifactName, NotebookUserAccess.artifactUserAccessRight AS AccessType,
                                NotebookUserAccess.displayName AS UserName, NotebookUserAccess.emailAddress AS UserEmail, NotebookUserAccess.PrincipalType,
                                NotebookUserAccess.userType, LastRefreshDatetime from NotebookUserAccess_view          
                              """)   
                    #display(result)

                    # For the first time, check if the table is present and create it, if not. else insert into the table
                    if spark.catalog.tableExists("WorkspaceAccessMetadata"):
                        result.write.format("delta").mode("append").save(myTablePath)
                        print("Table present, Data inserted")
                    else:
                        # Insert metadata to final table - WorkspaceAccessMetadata
                        result.write.format("delta").mode("overwrite").save(myTablePath)
                        print("Table created")

        if ("Lakehouse" not in df_viewcolumns.columns):
            pass
        else:
            print('Lakehouse column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["Lakehouse"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('Lakehouse column has value')    

                    # explode Lakehouse metadata
        	
                    result = spark.sql("""
                               create or replace temp view LakehouseMetadata_view AS
                               SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                               isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                               explode(Lakehouse) AS LakehouseMetadata FROM WorkspaceMetadata_view
                            """)

                    # Create uniform format
                    result = spark.sql("""
                                create or replace temp view LakehouseUserAccess_view AS 
                                Select Lakehousemetadata_view.*, LakehouseMetadata.name AS ArtifactName, explode(LakehouseMetadata.users) as LakehouseUserAccess from Lakehousemetadata_view
                            """)

                    result = spark.sql("""
                                SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                                WorkspaceState,'Lakehouse' AS ArtifactType, ArtifactName, LakehouseUserAccess.artifactUserAccessRight AS AccessType,
                                LakehouseUserAccess.displayName AS UserName, LakehouseUserAccess.emailAddress AS UserEmail, LakehouseUserAccess.PrincipalType,
                                LakehouseUserAccess.userType, LastRefreshDatetime from LakehouseUserAccess_view          
                            """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("DataPipeline" not in df_viewcolumns.columns):
            pass
        else:
            print('DataPipeline column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["DataPipeline"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('DataPipeline column has value')    
            
                    # explode DataPipeline metadata
        	
                    result = spark.sql("""
                                create or replace temp view DataPipelineMetadata_view AS
                                SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                                isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                                explode(DataPipeline) AS DataPipelineMetadata FROM WorkspaceMetadata_view
                            """)

                    # Create uniform format
                    result = spark.sql("""
                                create or replace temp view DataPipelineUserAccess_view AS 
                                Select DataPipelinemetadata_view.*, DataPipelineMetadata.name AS ArtifactName, 
                                explode(DataPipelineMetadata.users) as DataPipelineUserAccess from DataPipelinemetadata_view
                            """)

                    result = spark.sql("""
                                SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                                WorkspaceState,'DataPipeline' AS ArtifactType, ArtifactName, DataPipelineUserAccess.artifactUserAccessRight AS AccessType,
                                DataPipelineUserAccess.displayName AS UserName, DataPipelineUserAccess.emailAddress AS UserEmail, DataPipelineUserAccess.PrincipalType,
                                DataPipelineUserAccess.userType, LastRefreshDatetime from DataPipelineUserAccess_view          
                            """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("Eventstream" not in df_viewcolumns.columns):
            pass
        else:
            print('Eventstream column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["Eventstream"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('Eventstream column has value')    

                    # explode Eventstream metadata
        	
                    result = spark.sql("""
                            create or replace temp view EventstreamMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(Eventstream) AS EventstreamMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view EventstreamUserAccess_view AS 
                            Select Eventstreammetadata_view.*, EventstreamMetadata.name AS ArtifactName, explode(EventstreamMetadata.users) as EventstreamUserAccess from Eventstreammetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'Eventstream' AS ArtifactType, ArtifactName, EventstreamUserAccess.artifactUserAccessRight AS AccessType,
                           EventstreamUserAccess.displayName AS UserName, EventstreamUserAccess.emailAddress AS UserEmail, EventstreamUserAccess.PrincipalType,
                           EventstreamUserAccess.userType, LastRefreshDatetime from EventstreamUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("KQLDatabase" not in df_viewcolumns.columns):
            pass
        else:
            print('KQLDatabase column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["KQLDatabase"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('KQLDatabase column has value')    


                    # explode KQLDatabase metadata
        	
                    result = spark.sql("""
                                create or replace temp view KQLDatabaseMetadata_view AS
                                SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                                isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                                explode(KQLDatabase) AS KQLDatabaseMetadata FROM WorkspaceMetadata_view
                            """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view KQLDatabaseUserAccess_view AS 
                            Select KQLDatabasemetadata_view.*, KQLDatabaseMetadata.name AS ArtifactName, explode(KQLDatabaseMetadata.users) as KQLDatabaseUserAccess from KQLDatabasemetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'KQLDatabase' AS ArtifactType, ArtifactName, KQLDatabaseUserAccess.artifactUserAccessRight AS AccessType,
                           KQLDatabaseUserAccess.displayName AS UserName, KQLDatabaseUserAccess.emailAddress AS UserEmail, KQLDatabaseUserAccess.PrincipalType,
                           KQLDatabaseUserAccess.userType, LastRefreshDatetime from KQLDatabaseUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("KQLQueryset" not in df_viewcolumns.columns):
            pass
        else:
            print('KQLQueryset column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["KQLQueryset"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('KQLQueryset column has value')    

                    # explode KQLQueryset metadata
        	
                    result = spark.sql("""
                            create or replace temp view KQLQuerysetMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(KQLQueryset) AS KQLQuerysetMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view KQLQuerysetUserAccess_view AS 
                            Select KQLQuerysetmetadata_view.*, KQLQuerysetMetadata.name AS ArtifactName, explode(KQLQuerysetMetadata.users) as KQLQuerysetUserAccess from KQLQuerysetmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'KQLQueryset' AS ArtifactType, ArtifactName, KQLQuerysetUserAccess.artifactUserAccessRight AS AccessType,
                           KQLQuerysetUserAccess.displayName AS UserName, KQLQuerysetUserAccess.emailAddress AS UserEmail, KQLQuerysetUserAccess.PrincipalType,
                           KQLQuerysetUserAccess.userType, LastRefreshDatetime from KQLQuerysetUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("MLExperiment" not in df_viewcolumns.columns):
            pass
        else:
            print('MLExperiment column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["MLExperiment"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('MLExperiment column has value')    

                    # explode MLExperiment metadata
        	
                    result = spark.sql("""
                            create or replace temp view MLExperimentMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(MLExperiment) AS MLExperimentMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view MLExperimentUserAccess_view AS 
                            Select MLExperimentmetadata_view.*, MLExperimentMetadata.name AS ArtifactName, explode(MLExperimentMetadata.users) as MLExperimentUserAccess from MLExperimentmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'MLExperiment' AS ArtifactType, ArtifactName, MLExperimentUserAccess.artifactUserAccessRight AS AccessType,
                           MLExperimentUserAccess.displayName AS UserName, MLExperimentUserAccess.emailAddress AS UserEmail, MLExperimentUserAccess.PrincipalType,
                           MLExperimentUserAccess.userType, LastRefreshDatetime from MLExperimentUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("MLModel" not in df_viewcolumns.columns):
            pass
        else:
            print('MLModel column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["MLModel"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('MLModel column has value')    

                    # explode MLModel metadata
        	
                    result = spark.sql("""
                            create or replace temp view MLModelMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(MLModel) AS MLModelMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view MLModelUserAccess_view AS 
                            Select MLModelmetadata_view.*, MLModelMetadata.name AS ArtifactName, explode(MLModelMetadata.users) as MLModelUserAccess from MLModelmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'MLModel' AS ArtifactType, ArtifactName, MLModelUserAccess.artifactUserAccessRight AS AccessType,
                           MLModelUserAccess.displayName AS UserName, MLModelUserAccess.emailAddress AS UserEmail, MLModelUserAccess.PrincipalType,
                           MLModelUserAccess.userType, LastRefreshDatetime from MLModelUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("SQLAnalyticsEndpoint" not in df_viewcolumns.columns):
            pass
        else:
            print('SQLAnalyticsEndpoint column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["SQLAnalyticsEndpoint"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('SQLAnalyticsEndpoint column has value')    

                    # explode SQLAnalyticsEndpoint metadata
        	
                    result = spark.sql("""
                            create or replace temp view SQLAnalyticsEndpointMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(SQLAnalyticsEndpoint) AS SQLAnalyticsEndpointMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view SQLAnalyticsEndpointUserAccess_view AS 
                            Select SQLAnalyticsEndpointmetadata_view.*, SQLAnalyticsEndpointMetadata.name AS ArtifactName, explode(SQLAnalyticsEndpointMetadata.users) as SQLAnalyticsEndpointUserAccess from SQLAnalyticsEndpointmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'SQLAnalyticsEndpoint' AS ArtifactType, ArtifactName, SQLAnalyticsEndpointUserAccess.datamartUserAccessRight AS AccessType,
                           SQLAnalyticsEndpointUserAccess.displayName AS UserName, SQLAnalyticsEndpointUserAccess.emailAddress AS UserEmail, SQLAnalyticsEndpointUserAccess.PrincipalType,
                           SQLAnalyticsEndpointUserAccess.userType, LastRefreshDatetime from SQLAnalyticsEndpointUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("SparkJobDefinition" not in df_viewcolumns.columns):
            pass
        else:
            print('SparkJobDefinition column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["SparkJobDefinition"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('SparkJobDefinition column has value')    

                    # explode SparkJobDefinition metadata
        	
                    result = spark.sql("""
                            create or replace temp view SparkJobDefinitionMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(SparkJobDefinition) AS SparkJobDefinitionMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view SparkJobDefinitionUserAccess_view AS 
                            Select SparkJobDefinitionmetadata_view.*, SparkJobDefinitionMetadata.name AS ArtifactName, explode(SparkJobDefinitionMetadata.users) as SparkJobDefinitionUserAccess from SparkJobDefinitionmetadata_view
                        """)

                    result = spark.sql("""
                            SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                            WorkspaceState,'SparkJobDefinition' AS ArtifactType, ArtifactName, SparkJobDefinitionUserAccess.artifactUserAccessRight AS AccessType,
                            SparkJobDefinitionUserAccess.displayName AS UserName, SparkJobDefinitionUserAccess.emailAddress AS UserEmail, SparkJobDefinitionUserAccess.PrincipalType,
                            SparkJobDefinitionUserAccess.userType, LastRefreshDatetime from SparkJobDefinitionUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("dashboards" not in df_viewcolumns.columns):
            pass
        else:
            print('dashboards column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["dashboards"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('dashboards column has value')    

                    # explode dashboards metadata
        	
                    result = spark.sql("""
                            create or replace temp view dashboardsMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(dashboards) AS dashboardsMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view dashboardsUserAccess_view AS 
                            Select dashboardsmetadata_view.*, dashboardsMetadata.displayName AS ArtifactName, explode(dashboardsMetadata.users) as dashboardsUserAccess from dashboardsmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'dashboards' AS ArtifactType, ArtifactName, dashboardsUserAccess.dashboardUserAccessRight AS AccessType,
                           dashboardsUserAccess.displayName AS UserName, dashboardsUserAccess.emailAddress AS UserEmail, dashboardsUserAccess.PrincipalType,
                           dashboardsUserAccess.userType, LastRefreshDatetime from dashboardsUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("dataflows" not in df_viewcolumns.columns):
            pass
        else:
            print('dataflows column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["dataflows"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('dataflows column has value')    

                    # explode dataflows metadata
        	
                    result = spark.sql("""
                            create or replace temp view dataflowsMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(dataflows) AS dataflowsMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view dataflowsUserAccess_view AS 
                            Select dataflowsmetadata_view.*, dataflowsMetadata.name AS ArtifactName, explode(dataflowsMetadata.users) as dataflowsUserAccess from dataflowsmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'dataflows' AS ArtifactType, ArtifactName, dataflowsUserAccess.dataflowUserAccessRight AS AccessType,
                           dataflowsUserAccess.displayName AS UserName, dataflowsUserAccess.emailAddress AS UserEmail, dataflowsUserAccess.PrincipalType,
                           dataflowsUserAccess.userType, LastRefreshDatetime from dataflowsUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        # Comment datamarts for now until we understand the metadata structure for datamarts
        # if ("datamarts" not in df_viewcolumns.columns):
        #     pass
        # else:
        #     print('datamarts column present')

        #     # explode datamarts metadata
        	
        #     result = spark.sql("""
        #             create or replace temp view datamartsMetadata_view AS
        #             SELECT id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
        #             isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
        #             explode(datamarts) AS datamartsMetadata FROM WorkspaceMetadata_view
        #             """)

        #    if result.isEmpty():
        #        pass
        #    else:

        #        # Create uniform format
        #        result = spark.sql("""
        #                 create or replace temp view datamartsUserAccess_view AS 
        #                 Select datamartsmetadata_view.*, datamartsMetadata.name AS ArtifactName, explode(datamartsMetadata.users) as datamartsUserAccess from datamartsmetadata_view
        #             """)

        #        result = spark.sql("""
        #             SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
        #                    WorkspaceState,'datamarts' AS ArtifactType, ArtifactName, datamartsUserAccess.datamartUserAccessRight AS AccessType,
        #                    datamartsUserAccess.displayName AS UserName, datamartsUserAccess.emailAddress AS UserEmail, datamartsUserAccess.PrincipalType,
        #                    datamartsUserAccess.userType from datamartsUserAccess_view          
        #             """)   
        #        #display(result)

        #        # Insert metadata to final table - WorkspaceAccessMetadata
        #        result.write.format("delta").mode("append").save(myTablePath)

        if ("datasets" not in df_viewcolumns.columns):
            pass
        else:
            print('datasets column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["datasets"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('datasets column has value')    

                    # explode datasets metadata
        	
                    result = spark.sql("""
                            create or replace temp view datasetsMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(datasets) AS datasetsMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view datasetsUserAccess_view AS 
                            Select datasetsmetadata_view.*, datasetsMetadata.name AS ArtifactName, explode(datasetsMetadata.users) as datasetsUserAccess from datasetsmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'datasets' AS ArtifactType, ArtifactName, datasetsUserAccess.datasetUserAccessRight AS AccessType,
                           datasetsUserAccess.displayName AS UserName, datasetsUserAccess.emailAddress AS UserEmail, datasetsUserAccess.PrincipalType,
                           datasetsUserAccess.userType, LastRefreshDatetime from datasetsUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("environment" not in df_viewcolumns.columns):
            pass
        else:
            print('environment column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["environment"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('environment column has value')    

                    # explode environment metadata
        	
                    result = spark.sql("""
                            create or replace temp view environmentMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(environment) AS environmentMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view environmentUserAccess_view AS 
                            Select environmentmetadata_view.*, environmentMetadata.name AS ArtifactName, explode(environmentMetadata.users) as environmentUserAccess from environmentmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'environment' AS ArtifactType, ArtifactName, environmentUserAccess.artifactUserAccessRight AS AccessType,
                           environmentUserAccess.displayName AS UserName, environmentUserAccess.emailAddress AS UserEmail, environmentUserAccess.PrincipalType,
                           environmentUserAccess.userType, LastRefreshDatetime from environmentUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("kustoeventhubdataconnection" not in df_viewcolumns.columns):
            pass
        else:
            print('kustoeventhubdataconnection column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["kustoeventhubdataconnection"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('kustoeventhubdataconnection column has value')    


                    # explode kustoeventhubdataconnection metadata
        	
                    result = spark.sql("""
                            create or replace temp view kustoeventhubdataconnectionMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(kustoeventhubdataconnection) AS kustoeventhubdataconnectionMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view kustoeventhubdataconnectionUserAccess_view AS 
                            Select kustoeventhubdataconnectionmetadata_view.*, kustoeventhubdataconnectionMetadata.name AS ArtifactName, explode(kustoeventhubdataconnectionMetadata.users) as kustoeventhubdataconnectionUserAccess from kustoeventhubdataconnectionmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'kustoeventhubdataconnection' AS ArtifactType, ArtifactName, kustoeventhubdataconnectionUserAccess.artifactUserAccessRight AS AccessType,
                           kustoeventhubdataconnectionUserAccess.displayName AS UserName, kustoeventhubdataconnectionUserAccess.emailAddress AS UserEmail, kustoeventhubdataconnectionUserAccess.PrincipalType,
                           kustoeventhubdataconnectionUserAccess.userType, LastRefreshDatetime from kustoeventhubdataconnectionUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("reflexproject" not in df_viewcolumns.columns):
            pass
        else:
            print('reflexproject column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["reflexproject"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('reflexproject column has value')    

                    # explode reflexproject metadata
        	
                    result = spark.sql("""
                            create or replace temp view reflexprojectMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(reflexproject) AS reflexprojectMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view reflexprojectUserAccess_view AS 
                            Select reflexprojectmetadata_view.*, reflexprojectMetadata.name AS ArtifactName, explode(reflexprojectMetadata.users) as reflexprojectUserAccess from reflexprojectmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'reflexproject' AS ArtifactType, ArtifactName, reflexprojectUserAccess.artifactUserAccessRight AS AccessType,
                           reflexprojectUserAccess.displayName AS UserName, reflexprojectUserAccess.emailAddress AS UserEmail, reflexprojectUserAccess.PrincipalType,
                           reflexprojectUserAccess.userType, LastRefreshDatetime from reflexprojectUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        if ("reports" not in df_viewcolumns.columns):
            pass
        else:
            print('reports column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["reports"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('reports column has value')    

                    # explode reports metadata
        	
                    result = spark.sql("""
                            create or replace temp view reportsMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(reports) AS reportsMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view reportsUserAccess_view AS 
                            Select reportsmetadata_view.*, reportsMetadata.name AS ArtifactName, explode(reportsMetadata.users) as reportsUserAccess from reportsmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'reports' AS ArtifactType, ArtifactName, reportsUserAccess.reportUserAccessRight AS AccessType,
                           reportsUserAccess.displayName AS UserName, reportsUserAccess.emailAddress AS UserEmail, reportsUserAccess.PrincipalType,
                           reportsUserAccess.userType, LastRefreshDatetime from reportsUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)

        # if ("users" not in df_viewcolumns.columns):
        #     pass
        # else:
        #     print('users column present')

        #     # explode users metadata
        	
        #     result = spark.sql("""
        #             create or replace temp view usersMetadata_view AS
        #             SELECT id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
        #             isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
        #             explode(users) AS usersMetadata FROM WorkspaceMetadata_view
        #             """)

        #     # Create uniform format
        #     result = spark.sql("""
        #                 create or replace temp view usersUserAccess_view AS 
        #                 Select usersmetadata_view.*, usersMetadata.displayName AS ArtifactName, explode(usersMetadata.users) as usersUserAccess from usersmetadata_view
        #             """)

        #     result = spark.sql("""
        #             SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
        #                    WorkspaceState,'users' AS ArtifactType, ArtifactName, usersUserAccess.groupUserAccessRight AS AccessType,
        #                    usersUserAccess.displayName AS UserName, usersUserAccess.emailAddress AS UserEmail, usersUserAccess.PrincipalType,
        #                    usersUserAccess.userType from usersUserAccess_view          
        #             """)   
        #     #display(result)

        #     # Insert metadata to final table - WorkspaceAccessMetadata
        #     result.write.format("delta").mode("append").save(myTablePath)

        if ("warehouses" not in df_viewcolumns.columns):
            pass
        else:
            print('warehouses column present')

            # Check the value of the field
            for row in df_viewcolumns.collect():
                field_value = row["warehouses"]
                #display(len(field_value))
                if len(field_value) == 0:
                   pass
                else:
                    print('warehouses column has value')    

                    # explode warehouses metadata
        	
                    result = spark.sql("""
                            create or replace temp view warehousesMetadata_view AS
                            SELECT date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss z") AS LastRefreshDatetime, id AS WorkspaceId, name AS WorkspaceName, defaultDatasetStorageFormat AS WorkspaceDefaultDatasetStorageFormat, 
                            isOnDedicatedCapacity AS isWorkspaceOnDedicatedCapacity, state AS WorkspaceState, 
                            explode(warehouses) AS warehousesMetadata FROM WorkspaceMetadata_view
                        """)

                    # Create uniform format
                    result = spark.sql("""
                            create or replace temp view warehousesUserAccess_view AS 
                            Select warehousesmetadata_view.*, warehousesMetadata.name AS ArtifactName, explode(warehousesMetadata.users) as warehousesUserAccess from warehousesmetadata_view
                        """)

                    result = spark.sql("""
                           SELECT WorkspaceId, WorkspaceName, WorkspaceDefaultDatasetStorageFormat, isWorkspaceOnDedicatedCapacity, 
                           WorkspaceState,'warehouses' AS ArtifactType, ArtifactName, warehousesUserAccess.datamartUserAccessRight AS AccessType,
                           warehousesUserAccess.displayName AS UserName, warehousesUserAccess.emailAddress AS UserEmail, warehousesUserAccess.PrincipalType,
                           warehousesUserAccess.userType, LastRefreshDatetime from warehousesUserAccess_view          
                        """)   
                    #display(result)

                    # Insert metadata to final table - WorkspaceAccessMetadata
                    result.write.format("delta").mode("append").save(myTablePath)


StatementMeta(, 6e945aa3-363c-4cdf-9289-2cbdc3e9199b, 45, Finished, Available)

Workspaceid: 4b63d48a-82f6-4f27-bb75-7a1b8b22df7e


'Scanid for Workspaceid 4b63d48a-82f6-4f27-bb75-7a1b8b22df7e is fd2749f8-9410-4bf1-b117-acf39ca3c702'

'Status for Scanid fd2749f8-9410-4bf1-b117-acf39ca3c702 for Workspaceid 4b63d48a-82f6-4f27-bb75-7a1b8b22df7e is Succeeded'

'Scan for Scanid: fd2749f8-9410-4bf1-b117-acf39ca3c702 is complete'

...

'Scan for Scanid: f41eb0a4-d86c-463f-9484-ea04239f4560 is complete'

Valid workspace - do the processing
notebook column present
notebook column has value
Table present, Data inserted
Lakehouse column present
Lakehouse column has value
SQLAnalyticsEndpoint column present
SQLAnalyticsEndpoint column has value
dashboards column present
dataflows column present
datasets column present
datasets column has value
reports column present
Workspaceid: e26a57cd-fa71-4b01-8ea3-6190d4be4369


'Scanid for Workspaceid e26a57cd-fa71-4b01-8ea3-6190d4be4369 is 55d73d23-5e8e-440e-a20e-4458cba1d3cd'

'Status for Scanid 55d73d23-5e8e-440e-a20e-4458cba1d3cd for Workspaceid e26a57cd-fa71-4b01-8ea3-6190d4be4369 is Succeeded'

'Scan for Scanid: 55d73d23-5e8e-440e-a20e-4458cba1d3cd is complete'

# Change this to code cell when you need to delete data from the Metadata table 
delete_stmt = 'DELETE FROM '+myLakehouse+'.WorkspaceAccessMetadata'
result = spark.sql(delete_stmt)

%%sql
drop table lakehouse01.WorkspaceAccessMetadata

%%sql
select * from WorkspaceMetadata_view

display(df_scanResults)